In [ ]:
!pip install -qU rich
!pip install -qU wandb
!pip install -qU git+https://github.com/wandb/weave.git@feat/groq
!pip install -qU llama-index
!pip install -qU llama-index-embeddings-huggingface llama-index-llms-groq

In [ ]:
import os
import rich
from google.colab import userdata

import wandb
import weave

from llama_index.core import (
    ServiceContext, StorageContext, load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq

In [ ]:
wandb.init(project="groq-rag", job_type="query")
weave.init(project_name="groq-rag")

config = wandb.config
config.embed_model_name = "BAAI/bge-small-en-v1.5"
config.groq_llm = "mixtral-8x7b-32768"
config.similarity_top_k = 10

artifact = wandb.use_artifact(
    "geekyrakshit/groq-rag/ncert-flamingoes-prose-embeddings:latest",
    type="vector_index"
)
artifact_dir = artifact.download()

In [ ]:
llm = Groq(model=config.groq_llm, api_key=userdata.get("GROQ_API_KEY"))
embed_model = HuggingFaceEmbedding(model_name=config.embed_model_name)
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, llm=llm
)

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir=artifact_dir)
index = load_index_from_storage(
    storage_context, service_context=service_context
)

In [ ]:
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=config.similarity_top_k,
    # node_postprocessors=[cohere_rerank],
)

In [ ]:
query = """In the story \"The Last Lesson\",
what was the mood in the classroom when M. Hamel gave his last French lesson?
"""
response = query_engine.query(query).response

rich.print(response)

In [ ]:
table = wandb.Table(columns=["query", "response"], data=[[query, response]])
wandb.log({"response": table})

In [ ]:
wandb.finish()